# Question2 

- Analysis of data from nytimes api

In [1]:
import json, requests
from pprint import pprint
import datetime
import os
import glob
import nltk

api_key = os.getenv('auth_key')   #exporting the auth key stored
print(api_key)

path='..\\data\\nytimes'
if not os.path.exists(path): #making the folder in location if folder doesn't exist to store json files
    os.mkdir(path)

9d57a1c7179947cfaf6d7e89ac7234b3


In [ ]:
def parse_articles(articles):         #function to parse data to get only the required data to analyze
    data_result = []
    reqdata = {}
    for i in articles['response']['docs']:
        if i['abstract'] is not None:
                reqdata['abstract'] = i['abstract']
        reqdata['headline'] = i['headline']['main']
        if i['snippet'] is not None:
                reqdata['snippet'] = i['snippet']
        subjects = []
        for m in range(0,len(i['keywords'])):
                if 'subject' in i['keywords'][m]['name']:
                     subjects.append(i['keywords'][m]['value'])
        reqdata['subjects'] = subjects   
        data_result.append(reqdata)
    return data_result

In [ ]:
url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'

def get_articles(startDate,endDate,sort,page):   #function to get articles for each page from start date to end data using url
    q='Bill Clinton'
    fq = {'headline':'Bill Clinton', 'source':['The New York Times']}
    news=[]
    query_params={
              'begin_date': startDate,
              'end_date': endDate,
              'sort': sort,
              'q': q,
              'fq': fq,
              'page':str(page),
              'api-key': api_key
     }
    request=requests.get(url, params=query_params) 
    data=request.json()
    if data is not None:
        if 'response' in data:
              news.append(parse_articles(data))  #calling function to parse data and add it in list
        else:
            return         
    else:
        return            
    return news

In [ ]:
def get_everyYear(date):               #function to get all data of year passed iterating from page 0 to 100
    all_content=[]
    start = date + '0101',
    end = date + '1231',
    sort = 'newest'
    for page in range(0,100):
            content = get_articles(start,end,sort,page) #call punction to get data for each page from 0 to 100
            if content:
                all_content = all_content + content 
    return all_content    

In [ ]:
all_articles =[]            #Initializing list to store all articles 

for i in range(1887,2017):
    news = get_everyYear(str(i))  #Getting results of all articles from 1987 to 2017
    all_articles = all_articles + news  #updating all_articles list to get total list
    
    filePath=os.path.join(path,str(i))+'.json'   #path to store all json files of each year
    try:
         with open(filePath, 'w') as json_file:
                json.dump(all_articles,json_file,indent=2)   #dumping data in the file
    except ValueError:                                      
         print('Decoding JSON has failed')
json_file.close()



In [10]:
subjects =[]                  #Initialize list to store all subjects from year 1887 to 1997 referring Bill Clinton
clintonName = {}              #Initialize dict to store frequency of his name each year to know his popularity
elections={}                  #Initialize dict to store elctions data of each year

for infile in glob.glob(os.path.join(path, '*.json')):  #Iterating over all the json files stored in path
        year = infile[16:20]
        with open(infile) as filedata:               #opening each json file stored in path    
                d1 = json.load(filedata)             #getting the json data 
                for i in d1:
                    for j in i:                      #iterating over each json data
                        subjects.append(j['subjects'])    #appending all subjects in list
                        if any(('PRESIDENTIAL ELECTION OF ' + year) in s.upper() for s in j['subjects']):
                            elections[year]=j['snippet']
                        if 'Clinton' in j['snippet']:   #checking if clinton name mentioned in each json data
                            if year in clintonName:
                                clintonName[year]+=1
                            else:
                                clintonName[year]=1
        filedata.close()  
        


In [11]:
print('Clinton name mentioned')        
for year in sorted(clintonName.keys()):   #getting number of times Clinton name mentioned in each year
    print (year, clintonName[year])

# Analysis 1:Clinton joined politics for president elections from 1991 
#     and there was no much mention about him before 1992 and was most popular from year 1992-1995.
#He was mentioned in 2016 alot because of Hillary Clinton in elections       

Clinton name mentioned
1992 20
1993 20
1994 20
1995 30
1996 5
1997 5
1998 5
1999 5
2000 13
2001 5
2002 5
2003 5
2004 5
2005 5
2006 7
2007 7
2008 7
2009 8
2010 8
2011 10
2013 10
2014 8
2015 8
2016 12


In [12]:
all_subjects=[]      #Initializing list to store all subjects from json data

for m in subjects:
    for n in m:
        all_subjects.append(n.upper())   #storing all subjects in the list

words_freq={}

for i in all_subjects:                   #calculating the frequency of each subject
        if i in words_freq:
            words_freq[i]+=1
        else:
            words_freq[i]=1
            
top_words = sorted(words_freq, key=lambda k: words_freq[k], reverse=True) #sorting the subject based on freq in decending order


In [13]:
for i in top_words:
    print(i ,':' ,words_freq[i])

# Analysis2: Bill Clinton was a political person in USA as maximum time name was mentioned in politics subject,
# and his frequency of reference in all subjects.   

UNITED STATES POLITICS AND GOVERNMENT : 97
LAW AND LEGISLATION : 75
ELECTIONS : 68
UNITED STATES INTERNATIONAL RELATIONS : 68
HEALTH INSURANCE : 43
MEDICINE AND HEALTH : 43
ELECTION ISSUES : 40
POLITICS AND GOVERNMENT : 39
PRESIDENTIAL ELECTION OF 1992 : 37
BIOGRAPHICAL INFORMATION : 36
TELEVISION : 35
PRESIDENTIAL ELECTION OF 2012 : 32
REFORM AND REORGANIZATION : 30
UNITED STATES ARMAMENT AND DEFENSE : 27
NEWS AND NEWS MEDIA : 27
FINANCES : 26
UNITED STATES ECONOMY : 24
STATES (US) : 24
ECONOMIC CONDITIONS AND TRENDS : 23
PRESIDENTIAL ELECTION OF 2004 : 21
LABOR : 21
WHITEWATER CASE : 20
LABOR-MANAGEMENT RELATIONS : 20
EDUCATION (SECONDARY) : 20
CORRECTION STORIES : 20
HEALTH MAINTENANCE ORGANIZATIONS : 20
TUTORS AND TUTORING : 20
KUWAIT-INTERNATIONAL RELATIONS-US : 20
MEDICAID : 20
LEGAL PROFESSION : 20
BOOKS AND LITERATURE : 20
HAITI-INTERNATIONAL RELATIONS-US : 20
VETERANS : 16
ENVIRONMENT : 16
PRESIDENTIAL ELECTION OF 2000 : 16
PREGNANCY AND OBSTETRICS : 15
ETHICS : 15
PRESIDENTIA

In [14]:
for i in range(1991,2017):
    sub='PRESIDENTIAL ELECTION OF ' + str(i)
    if sub in set(all_subjects) :
        print(i)
        print(elections[str(i)])


1992
Hours before last night's debate, the Bush organization began broadcasting its first commercial devoted entirely to questioning Gov. Bill Clinton's credibility on the draft. Bush aides said the 30-second ad would appear for several days in select ...
1996
To the Editor:    Once again the strange workings of our Electoral College have confounded pundits and misled headline writers. You say (front page, Nov. 6) that Bill Clinton won ''with solid margins across U.S.'' Other newspapers proclaim a ''big...
2000
Memo to the half-naked teenager waving the Bradley poster on the corner in Concord: It's winter in New England. Bill wants you to put your shirt back on.    This New Hampshire primary has been wonderful. For one brief moment, in one smallish state...
2004
At a meeting of Democrats in Parkersburg, Iowa, Senator John Edwards is listening closely as a woman describes her struggle to afford a nursing home for her ailing mother. ''I want somebody who really knows what it is like to s

In [15]:
#Analysis 3: Elections were held on these many years in US from 1991-2017 as it was mentioned in subjects.
#1992-2000-Bush and Bill Clinton name was mentioned in news for presidential election
#2012-Barack Obama name was there in presidential election.
#2016-Donald Trump name was in news for presidential election.